In [1]:
import pandas as pd
import os
import git
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics

def get_git_root(path):

        git_repo = git.Repo(path, search_parent_directories=True)
        #git_root = git_repo.git.rev_parse("--show-toplevel")
        
        return git_repo.working_dir

In [91]:
top_level_git_dir = get_git_root(os.getcwd())
raw_data_dir = os.path.join(top_level_git_dir, "data", "raw")

train_csv_path = os.path.join(raw_data_dir, "train_values.csv")
test_csv_path = os.path.join(raw_data_dir, "test_values.csv")
train_labels_csv_path = os.path.join(raw_data_dir, "train_labels.csv")
submission_format_csv_path = os.path.join(raw_data_dir, "submission_format.csv")

train_df = pd.read_csv(train_csv_path, index_col = "row_id")
train_labels_df = pd.read_csv(train_labels_csv_path, index_col = "process_id")
test_df = pd.read_csv(test_csv_path, index_col = "row_id")
submission_format_df = pd.read_csv(submission_format_csv_path, index_col = "process_id")


/Users/greg.bolla/.local/share/virtualenvs/driven-data-rinse-over-run-w5STMFPe/lib/python3.6/site-packages/numpy/lib/arraysetops.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def prep_full_df(df):

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    df = df.assign(turbidity_in_liters = \
        np.maximum(0, df.return_flow) * df.return_turbidity)

    df['process_phase'] = df.process_id.astype(str) + "_" + df.phase.astype(str)
    df = df[df.phase != "final_rinse"]
    
    return df

In [4]:
def prep_metadata(df):
    meta_df = df[["process_id", "pipeline"]].drop_duplicates().set_index("process_id")
    meta_df = pd.get_dummies(meta_df)
    
    if 'L12' not in meta_df.columns:
        meta_df['pipeline_L12'] = False
    
    for col in meta_df.columns:
        if "pipeline" in col:
            meta_df[col] = meta_df[col].astype(bool)
            
    meta_df["num_phases"] = df.groupby("process_id")["phase"].apply(lambda x: x.nunique())
    
    return meta_df

In [5]:
ts_cols = [
    'process_id',
    'timestamp',
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid',
    "turbidity_in_liters"
]

def prep_time_series_features(df, columns = None):
    
    if columns is None:
        columns = df.columns
    
    df = df.sort_values(by=["process_id", "timestamp"], ascending=True)
    process_duration_ts = df.groupby('process_id')["timestamp"].max() - df.groupby('process_id')["timestamp"].min() 
    process_duration_ts = process_duration_ts.rename('process_duration')
    process_duration = process_duration_ts.apply(lambda row: row.total_seconds())
    
    ts_df = df[ts_cols].set_index('process_id')
    
    # define fxn before calling in .agg to make col name more descriptive (in place of <lambda>)
    def last_five_mean(x):
        return x.tail(5).mean()
    
    ts_features_agg_df = ts_df.groupby('process_id').agg(['min', 'max', 'mean', 'std', last_five_mean])
    
    ts_features_df = pd.concat([process_duration, ts_features_agg_df], axis = 1)
    return ts_features_df

In [6]:
def prep_dummy_vars(df):
    
    categorical_cols = ["num_phases"]

    for cat_col in categorical_cols:
        dummy_df = pd.get_dummies(df[cat_col], prefix=cat_col, dummy_na = False)
        dummy_df = dummy_df.astype('bool')

        drop_val = df.groupby([cat_col]).size().idxmax()

        drop_col = "{}_{}".format(cat_col, drop_val)
        df = pd.concat([df, dummy_df], axis=1)
        df = df.drop(drop_col, axis=1)    
        df[cat_col] = df[cat_col].astype(object)
    
    return df

In [7]:
def clean_feature_df(df):
    
    new_col_names = []
    for col in df.columns.ravel():
        if isinstance(col, str):
            new_col_names.append(col)
        elif isinstance(col, tuple):
            col_name = "{}_{}".format(col[0], col[1])
            new_col_names.append(col_name)
    df.columns = new_col_names
    
    return df

In [8]:
def create_feature_matrix(df):
    
    prepped_df = prep_full_df(df)
    metadata_df = prep_metadata(prepped_df)
    time_series_df = prep_time_series_features(prepped_df)
    
    dfs_to_concat = [metadata_df, time_series_df]
    
    feature_df = pd.concat(dfs_to_concat, axis=1)
    feature_df = prep_dummy_vars(feature_df)
    
    df_to_return = clean_feature_df(feature_df)

    return df_to_return

In [9]:
train_features_df = create_feature_matrix(train_df)

indices_to_keep = list(set(train_features_df.index).intersection(set(train_labels_df.index)))
# figure out why 16 indices dropped out of train_features_df
train_labels_df = train_labels_df[train_labels_df.index.isin(indices_to_keep)]
train_features_w_response = train_features_df.join(train_labels_df)
train_features_w_response.head()

,pipeline_L1,pipeline_L10,pipeline_L11,pipeline_L12,pipeline_L2,pipeline_L3,pipeline_L4,pipeline_L6,pipeline_L7,pipeline_L8,...,tank_concentration_acid_last_five_mean,turbidity_in_liters_min,turbidity_in_liters_max,turbidity_in_liters_mean,turbidity_in_liters_std,turbidity_in_liters_last_five_mean,num_phases_1,num_phases_2,num_phases_3,final_rinse_total_turbidity_liter
process_id,,,,,,,,,,,,,,,,,,,,,
20001,False,False,False,False,False,False,True,False,False,False,...,44.653038,818.406942,1.579919e+06,105488.460366,174650.861160,30300.051942,False,False,False,4.318275e+06
20002,False,False,False,False,False,True,False,False,False,False,...,44.229616,499.442792,2.976941e+06,854203.372900,563689.439444,949644.159635,False,True,False,4.375286e+05
20003,False,False,False,False,False,True,False,False,False,False,...,44.716846,152.522484,1.431140e+06,44218.000816,127420.220308,5287.641592,False,False,False,4.271977e+05
20004,False,False,False,False,False,False,False,False,True,False,...,45.226021,0.000000,3.162818e+06,212923.854423,387856.686586,22306.533910,False,False,False,7.197830e+05
20005,False,False,False,False,False,False,False,False,True,False,...,43.952939,0.000000,2.066256e+05,23587.698324,26813.228206,45723.010454,True,False,False,4.133107e+05


In [46]:
response_var = ["final_rinse_total_turbidity_liter"]
pred_df = train_features_w_response.drop(response_var, axis=1)
response_df = train_features_w_response[response_var]

pred_train, pred_test, response_train, response_test = train_test_split(pred_df, response_df, test_size=0.01, random_state=223)



In [11]:
def adj_r2_score(lm, y, y_pred):
    adj_r2 = 1 - float(len(y)-1)/(len(y)-len(lm.coef_)-1)*(1 - metrics.r2_score(y,y_pred))
    return adj_r2

In [77]:
def calc_mape(pred_array, actual_array):
    
    threshold = 290000

    
    mape_array = []
    for pred, actual in zip(pred_array, actual_array.values):
        #print("{} - {}".format(pred[0], type(pred[0])))
        #print("{} - {}".format(actual[0], type(actual[0])))
        mape = (abs(pred - actual) / max(abs(actual), threshold))
        mape_array.append(mape)
        
    return mape_array


In [78]:
single_regression_df = pd.DataFrame()

for col_name in pred_train.columns:
    categorical = True if pred_train[col_name].dtype == "object" else False

    full_df_col_list = pred_df.columns.values
    if categorical == True:
        col_vars = [var for var in full_df_col_list if "{}_".format(col_name) in var]
    else: 
        col_vars = [col_name]
        
    x_train_df = pred_train[col_vars]
    x_test_df = pred_test[col_vars]
    
    x_model = linear_model.LinearRegression()
    x_results = x_model.fit(x_train_df, response_train[response_var])
    
    y_predicted_test = x_model.predict(x_test_df)
    
    med_absolute_error = metrics.median_absolute_error(response_test[response_var], y_predicted_test)
    absolute_perc_error = calc_mape(y_predicted_test, response_test[response_var])
    r2 = metrics.r2_score(response_test[response_var],y_predicted_test)
    adj_r2 = adj_r2_score(x_results, response_test[response_var],y_predicted_test)
    
    coef_dict = {}
    model_rows = []

    for idx, col in enumerate(x_train_df.columns.values):
        #if categorical == True:
        #    print(col)
        #    print(x_results.coef_)
        #    pred_coef = x_results.coef_[0][idx]
        #else:
        #    pred_coef = x_results.coef_[idx]
        model_row = {}
        model_row['model'] = col_name
        model_row['model_r2'] = r2
        model_row['model_adj_r2'] = adj_r2
        model_row['median_absolute_error'] = med_absolute_error
        model_row['mean_abs_perc_error'] = np.mean(absolute_perc_error)
        model_row['pred_col'] = col if categorical == True else None
        #model_row['pred_coef'] = pred_coef
        model_row['categorical'] = categorical
        model_rows.append(model_row)
        
    single_regression_df = single_regression_df.append(model_rows, ignore_index=True)

In [68]:
x_results.coef_[0][0]

IndexError: invalid index to scalar variable.

In [14]:
for idx, col in enumerate(x_train_df.columns.values):
    print(idx)
    print(col)

0
num_phases_3


In [16]:
x_results.coef_

array([[-1093748.27478775]])

In [79]:
single_regression_df.to_csv("single_reg.csv", index=False)
single_regression_df.sort_values("mean_abs_perc_error", ascending=True).head(n =10)

,categorical,mean_abs_perc_error,median_absolute_error,model,model_adj_r2,model_r2,pred_col
19,False,1.588375,7.731755e+05,supply_flow_last_five_mean,0.016983,0.036644,None
17,False,1.639009,8.923325e+05,supply_flow_mean,-0.005141,0.014962,None
42,False,1.672209,8.704076e+05,return_flow_mean,-0.002725,0.017330,None
44,False,1.757559,9.231294e+05,return_flow_last_five_mean,-0.010571,0.009640,None
18,False,1.910872,1.183281e+06,supply_flow_std,-0.001550,0.018481,None
16,False,1.924231,1.177607e+06,supply_flow_max,-0.015943,0.004375,None
10,False,1.952890,1.135268e+06,pipeline_L9,0.139027,0.156246,None
5,False,1.963184,1.446873e+06,pipeline_L3,-0.022766,-0.002311,None
93,False,1.967938,1.382381e+06,turbidity_in_liters_std,-0.042130,-0.021288,None
90,False,2.037535,1.419319e+06,turbidity_in_liters_min,-0.030225,-0.009621,None


In [80]:
response_var = 'final_rinse_total_turbidity_liter'
# include best single reg variables
col_vars = [
    "supply_flow_last_five_mean", 
    "return_flow_mean",
    "pipeline_L9",
    "supply_flow_std",
    "return_flow_std",
]
multi_reg_model = linear_model.LinearRegression()
multi_reg_df_train = pred_train[col_vars]
multi_reg_df_test = pred_test[col_vars]

multi_reg_results = multi_reg_model.fit(multi_reg_df_train, response_train[response_var])

In [81]:
multi_reg_predicted_test = multi_reg_model.predict(multi_reg_df_test)

In [85]:
mape_test = calc_mape(multi_reg_predicted_test, response_test[response_var])
np.mean(mape_test)

1.4705834215786524

In [86]:
test_features_df = create_feature_matrix(test_df)

In [87]:
multi_reg_submissions_df = test_features_df[col_vars]
multi_reg_test_preds = multi_reg_model.predict(multi_reg_submissions_df)

In [93]:
assert np.all(test_features_df.index == submission_format_df.index), "index of submission doesn't match format"


In [94]:
multi_reg_submission = pd.DataFrame(data = multi_reg_test_preds,
                                 columns = submission_format_df.columns,
                                 index = submission_format_df.index)

In [95]:
multi_reg_submission.head()

,final_rinse_total_turbidity_liter
process_id,
20000,1.490514e+06
20006,7.681619e+05
20007,6.768452e+05
20009,1.015515e+06
20010,1.699212e+06


In [96]:
multi_reg_submission.to_csv(os.path.join(top_level_git_dir, "models", "multi_reg_submission_1.0.csv"))